
# 문제 1-2 : 2단계 Multi Chain — 여행지 정보 시스템

**목표**:  
1단계 — 여행지(도시/국가) 입력 → 대표 명소 1가지 추천  
2단계 — 추천 명소의 **역사/특징/방문 팁** 상세 정보 제공  

`ChatPromptTemplate`의 **system / user 메시지**를 사용하고, 두 체인을 LCEL로 연결합니다.


In [1]:

# 필수 패키지(처음 1회만)
# !pip install -U langchain-core langchain-openai python-dotenv


In [2]:

import os
from dotenv import load_dotenv, find_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI

env_path = find_dotenv(usecwd=True)
load_dotenv(env_path)

API_KEY = os.getenv("OPENAI_API_KEY") or os.getenv("GROQ_API_KEY")
if not API_KEY:
    raise RuntimeError(f".env에 OPENAI_API_KEY 또는 GROQ_API_KEY가 없습니다. 찾은 경로: {env_path or '(없음)'}")

print("[API prefix]", API_KEY[:6])

llm = ChatOpenAI(
    api_key=API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7,
)


[API prefix] gsk_Zl


In [3]:

# 1단계: 명소 추천
stage1_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 세계 여행 가이드 전문가입니다. 사용자가 말한 도시/국가에서 대표 명소 1가지를 정확하게 추천하세요. 고유명사 1개만 출력."),
    ("user", "여행지: {destination}\n추천 명소 1개만 출력:")
])
stage1_chain = stage1_prompt | llm | StrOutputParser()

# 2단계: 상세 정보
stage2_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행지 설명 전문가입니다. 명소의 역사, 특징, 방문 팁을 간결하게 한국어로 정리하세요."),
    ("user",
     "명소: {place}\n\n다음 형식을 지키세요:\n"
     "명소: {place}\n"
     "역사:\n- ...\n- ...\n"
     "특징:\n- ...\n- ...\n"
     "방문 팁:\n- ...\n- ...")
])
stage2_chain = stage2_prompt | llm | StrOutputParser()

def run_two_stages(inputs):
    place = stage1_chain.invoke({"destination": inputs["destination"]}).strip()
    detail = stage2_chain.invoke({"place": place})
    return {"stage1_place": place, "stage2_detail": detail}

pipeline = RunnablePassthrough() | RunnableLambda(run_two_stages)


In [4]:

destination = "로마"
result = pipeline.invoke({"destination": destination})
print("1단계(추천 명소):", result["stage1_place"])
print("\n2단계(상세 정보):\n" + result["stage2_detail"])


1단계(추천 명소): 콜로세움

2단계(상세 정보):
명소: 콜로세움

역사:
- 콜로세움은 로마 제국의 상징적인 건축물로, 이탈리아 로마에 위치해 있습니다.
- 72년부터 80년까지 로마 황제 베스파시아누스가 건설을 시작했으며, 그의 아들 티투스가 완성했습니다.

특징:
- 콜로세움은 원형 경기장으로, 최대 5만 명의 관중을 수용할 수 있었습니다.
- 검투사 경기, 동물 사냥, 공공 행사 등 다양한 이벤트가 이곳에서 열렸습니다.

방문 팁:
- 콜로세움은 로마의 중심부에 위치해 있어 대중교통을 이용하여 쉽게 접근할 수 있습니다.
- 방문객들은 콜로세움의 역사와 건축물에 대해 더 자세히 알기 위해 가이드 투어를 이용하는 것을 추천합니다.
